#### Image Dehazing code

Link of dataset : https://www.kaggle.com/datasets/rajat95gupta/hazing-images-dataset-cvpr-2019

In [4]:
"""Single image dehazing."""
from __future__ import division
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [95]:
# Code with intial code + tr map change
class Channel_value:
    val = -1.0
    intensity = -1.0


def find_intensity_of_atmospheric_light(img, gray):
    top_num = int(img.shape[0] * img.shape[1] * 0.001)
    toplist = [Channel_value()] * top_num
    dark_channel = find_dark_channel(img)
    l = [x.val for x in toplist]

    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            val = img.item(y, x, dark_channel)
            intensity = gray.item(y, x)
            toplist.sort(key = lambda x: x.val, reverse = True)

            if (-1 in l):
                i = l.index(-1)
                l[i] = val
                toplist[i].val = val
                toplist[i].intensity = intensity 
            else:
                for t in toplist:
                    if t.val < val or (t.val == val and t.intensity < intensity):
                        t.val = val
                        t.intensity = intensity
                        break

    toplist.sort(key = lambda x: x.intensity, reverse = True)

    return toplist[0].intensity


def find_dark_channel(img):
    return np.unravel_index(np.argmin(img), img.shape)[2]


def clamp(minimum, x, maximum):
    return max(minimum, min(x, maximum))

def create_fine_map(img, y, x):
    return min(img.item(y, x, 0), img.item(y, x, 1), img.item(y, x, 2))


def refining_coarse_map(img, sliceimg, y_c, x_c, windowSize):
    size = (sliceimg.shape[0], sliceimg.shape[1])
    max_coarse = -math.inf
    for y in range(size[0]):
        for x in range(size[1]):
            x_low = max(x-(windowSize//2), 0)
            y_low = max(y-(windowSize//2), 0)
            x_high = min(x+(windowSize//2), size[1])
            y_high = min(y+(windowSize//2), size[0])

            s = img[y_low:y_high, x_low:x_high]
            dark_channel = find_dark_channel(s)

            m_coarse = img.item(y, x, dark_channel)
            if m_coarse > max_coarse:
                max_coarse = m_coarse

    m_fine = create_fine_map(img, y_c, x_c)

    return min(max_coarse, m_fine)


def dehaze(img, windowSize, light_intensity, t0, w):
    size = (img.shape[0], img.shape[1])

    outimg = np.zeros(img.shape, img.dtype)

    for y in range(size[0]):
        for x in range(size[1]):
            x_low = max(x-(windowSize//2), 0)
            y_low = max(y-(windowSize//2), 0)
            x_high = min(x+(windowSize//2), size[1])
            y_high = min(y+(windowSize//2), size[0])

            sliceimg = img[y_low:y_high, x_low:x_high]

            m_t = refining_coarse_map(img, sliceimg, y, x, windowSize)
            t = 1.0 - (w * m_t/ light_intensity)

            outimg.itemset((y,x,0), clamp(0, ((img.item(y,x,0) - light_intensity) / max(t, t0) + light_intensity), 255))
            outimg.itemset((y,x,1), clamp(0, ((img.item(y,x,1) - light_intensity) / max(t, t0) + light_intensity), 255))
            outimg.itemset((y,x,2), clamp(0, ((img.item(y,x,2) - light_intensity) / max(t, t0) + light_intensity), 255))
    return outimg

In [ ]:
def main():
    image_name = '48_hazy'
    image_gt_name = '48_GT'
    img = cv2.imread(str(image_name + '.png'))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    light_intensity = find_intensity_of_atmospheric_light(img, gray)
    w = 0.99
    t0 = 0.55
    outimg = dehaze(img, 20, light_intensity, t0, w)
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    plt.subplot(131)
    plt.imshow(img)
    plt.subplot(132)
    plt.imshow(outimg)

    cv2.imwrite(str('out_' + image_name + '.png'), outimg)

    img_clear = cv2.imread(str(image_gt_name + '.png'))
    img_clear_gray = cv2.cvtColor(img_clear, cv2.COLOR_BGR2GRAY)

    img_hazed = cv2.imread(str(image_name + '.png'))
    img_hazed_gray = cv2.cvtColor(img_hazed, cv2.COLOR_BGR2GRAY)

    img_dehazed = cv2.imread(str('out_' + image_name + '.png'))
    img_dehazed_gray = cv2.cvtColor(img_dehazed, cv2.COLOR_BGR2GRAY)

    cl_de = ssim(img_clear_gray, img_dehazed_gray)
    print('Clear image vs Dehazed image', cl_de)

    cl_haz = ssim(img_clear_gray, img_hazed_gray)
    print('Clear image vs Hazed image', cl_haz)

if __name__ == "__main__": main()

In [30]:
# 33_haze : Haze removal with window size 300 : Comparison with clear image
from skimage.metrics import structural_similarity as ssim

img_dehazed = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/33_hazy_window300_w0.99.png')
img_dehazed_gray = cv2.cvtColor(img_dehazed, cv2.COLOR_BGR2GRAY)

img_hazed = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/GT/33_GT.png')
img_hazed_gray = cv2.cvtColor(img_hazed, cv2.COLOR_BGR2GRAY)


ssim(img_hazed_gray, img_dehazed_gray)

0.44937415757732235

In [31]:
# 33_haze : Haze removal with window size 100 : Comparison with clear image
img_dehazed = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/33_hazy_window100_w0.99.png')
img_dehazed_gray = cv2.cvtColor(img_dehazed, cv2.COLOR_BGR2GRAY)

ssim(img_hazed_gray, img_dehazed_gray)

0.44937415757732235

In [74]:
# 33_haze : Haze removal with window size 300 : Comparison with clear image
from skimage.metrics import structural_similarity as ssim

img_dehazed = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/out_33_hazy.png')
img_dehazed_gray = cv2.cvtColor(img_dehazed, cv2.COLOR_BGR2GRAY)

img_or = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/33_GT.png')
img_or_gray = cv2.cvtColor(img_or, cv2.COLOR_BGR2GRAY)

img_hazed = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/33_hazy.png')
img_hazed_gray = cv2.cvtColor(img_hazed, cv2.COLOR_BGR2GRAY)



ssim(img_or_gray, img_dehazed_gray)

0.44937415757732235

In [75]:
ssim(img_or_gray, img_hazed_gray)

0.35948850485634826

In [77]:
cv2.PSNR(img_or_gray, img_dehazed_gray)

15.000060116989083

In [78]:
cv2.PSNR(img_or_gray, img_hazed_gray)

9.956782885880157

In [79]:
# 33_haze : Haze removal with window size 300 : Comparison with clear image
from skimage.metrics import structural_similarity as ssim

img_dehazed = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/out_05_hazy.png')
img_dehazed_gray = cv2.cvtColor(img_dehazed, cv2.COLOR_BGR2GRAY)

img_or = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/05_GT.png')
img_or_gray = cv2.cvtColor(img_or, cv2.COLOR_BGR2GRAY)

img_hazed = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/05_hazy.png')
img_hazed_gray = cv2.cvtColor(img_hazed, cv2.COLOR_BGR2GRAY)



ssim(img_or_gray, img_dehazed_gray)

0.453828509438771

In [80]:
ssim(img_or_gray, img_hazed_gray)

0.36924787349451244

In [81]:
cv2.PSNR(img_or_gray, img_dehazed_gray)

11.630859703775124

In [82]:
cv2.PSNR(img_or_gray, img_hazed_gray)

8.81179318518615

In [83]:
# 33_haze : Haze removal with window size 300 : Comparison with clear image
from skimage.metrics import structural_similarity as ssim

img_dehazed = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/out_48_hazy.png')
img_dehazed_gray = cv2.cvtColor(img_dehazed, cv2.COLOR_BGR2GRAY)

img_or = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/48_GT.png')
img_or_gray = cv2.cvtColor(img_or, cv2.COLOR_BGR2GRAY)

img_hazed = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/48_hazy.png')
img_hazed_gray = cv2.cvtColor(img_hazed, cv2.COLOR_BGR2GRAY)



ssim(img_or_gray, img_dehazed_gray)

0.38804560663918497

In [84]:
ssim(img_or_gray, img_hazed_gray)

0.3512865230889368

In [85]:
cv2.PSNR(img_or_gray, img_dehazed_gray)

12.832523147050864

In [86]:
cv2.PSNR(img_or_gray, img_hazed_gray)

11.991906265353329

In [87]:
# 33_haze : Haze removal with window size 300 : Comparison with clear image
from skimage.metrics import structural_similarity as ssim

img_dehazed = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/out_33_hazy.png')
img_dehazed_gray = cv2.cvtColor(img_dehazed, cv2.COLOR_BGR2GRAY)

img_or = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/33_GT.png')
img_or_gray = cv2.cvtColor(img_or, cv2.COLOR_BGR2GRAY)

img_hazed = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/33_hazy.png')
img_hazed_gray = cv2.cvtColor(img_hazed, cv2.COLOR_BGR2GRAY)



ssim(img_or_gray, img_dehazed_gray)

0.44937415757732235

In [88]:
ssim(img_or_gray, img_hazed_gray)

0.35948850485634826

In [89]:
cv2.PSNR(img_or_gray, img_dehazed_gray)

15.000060116989083

In [90]:
cv2.PSNR(img_or_gray, img_hazed_gray)

9.956782885880157

In [91]:
# 33_haze : Haze removal with window size 300 : Comparison with clear image
from skimage.metrics import structural_similarity as ssim

img_dehazed = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/test_window300_w0.99.png')
img_dehazed_gray = cv2.cvtColor(img_dehazed, cv2.COLOR_BGR2GRAY)

img_or = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/04_GT.png')
img_or_gray = cv2.cvtColor(img_or, cv2.COLOR_BGR2GRAY)

img_hazed = cv2.imread('C:/Users/aditi/OneDrive/Desktop/NEU/NEU Fall 22 (Computer Vision)/Project/Data/results_defogging/Brightness Correction/demo scipt data/04_hazy.png')
img_hazed_gray = cv2.cvtColor(img_hazed, cv2.COLOR_BGR2GRAY)



ssim(img_or_gray, img_dehazed_gray)

0.19266414262326562

In [92]:
ssim(img_or_gray, img_dehazed_gray)

0.19266414262326562

In [93]:
cv2.PSNR(img_or_gray, img_dehazed_gray)

11.328586489992354

In [94]:
cv2.PSNR(img_or_gray, img_hazed_gray)

9.265462505101373